In [3]:
import opennmt
import os
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
import numpy as np
import sacrebleu
import pyonmttok
from opennmt.utils import checkpoint as checkpoint_util
from pyonmttok import SentencePieceTokenizer

In [2]:
def count_weights(model):
  trainable_count = np.sum([K.count_params(w) for w in model.trainable_weights])
  non_trainable_count = np.sum([K.count_params(w) for w in model.non_trainable_weights])

  print('Total params: {:,}'.format(trainable_count + non_trainable_count))
  print('Trainable params: {:,}'.format(trainable_count))
  print('Non-trainable params: {:,}'.format(non_trainable_count))

def compute_scores(runner, features_filename, labels_filename, pred_filename, include_ppl=False, include_ter=False):
  runner.infer(features_filename, pred_filename)

  dot_idx = pred_filename.index('.')
  base_pred_name = pred_filename[0:dot_idx]
  dot_idx = labels_filename.index('.')
  base_model_name = labels_filename[0:dot_idx]
  pred_filename = detokenize_data(base_pred_name, base_model_name)
  detokenized_labels_filename = detokenize_data(base_model_name, base_model_name)
  preds = []
  truth = []
  with open(pred_filename) as f:
    preds = f.readlines()

  with open(detokenized_labels_filename) as f:
    truth = f.readlines()

  scores = dict()
  if include_ppl:
    scores = runner.evaluate(
        features_file=features_filename,
        labels_file=labels_filename)
  
  bleu = sacrebleu.corpus_bleu(preds, [truth])
  scores.update({'bleu': bleu.score})
  if include_ter:
    ter = sacrebleu.corpus_ter(preds, [truth])
    scores.update({'ter': ter.score})
  
  return scores

def detokenize(model_basename, tokenized_basename):
  model_path = os.path.join("sentencepiece_models", model_basename + ".model")
  vocabulary_path = os.path.join("sentencepiece_models", f"{model_basename}.vocab")
  detokenizer = SentencePieceTokenizer(model_path=model_path,
                                     vocabulary_path=vocabulary_path,)
  
  with open(f"{tokenized_basename}.tok") as f:
    with open(f"{tokenized_basename}.txt", mode="w") as fout:
      for line in f.readlines():
        fout.write(detokenizer.detokenize(line.strip().split(" ")) + "\n")

  return f"{tokenized_basename}.txt"

def tokenize(input_file, basename):
  model_path = os.path.join("sentencepiece_models", f"{basename}.model")
  vocabulary_path = os.path.join("sentencepiece_models", f"{basename}.vocab")
  tokenizer = SentencePieceTokenizer(model_path=model_path,
                                     vocabulary_path=vocabulary_path,)
  
  with open(os.path.join(f"{input_file}")) as f:
    with open(os.path.join(f"{basename}.tok"), mode="w") as fout:
      for line in f.readlines():
        if line.strip():
          fout.write(" ".join(tokenizer.tokenize(line)[0]) + "\n")

In [7]:
# Build vocab (uses SentencePiece)
# source = catalan   (ca)
# pivot  = spanish   (es)
# target = italian  (it)

!onmt-build-vocab --sentencepiece model_type=bpe --size 32000 --save_vocab sentencepiece_models/src src_pvt_data/src_train.txt
!onmt-build-vocab --sentencepiece model_type=bpe --size 32000 --save_vocab sentencepiece_models/pvt_src src_pvt_data/pvt_src_train.txt

!onmt-build-vocab --sentencepiece model_type=bpe --size 32000 --save_vocab sentencepiece_models/tgt pvt_tgt_data/tgt_train.txt
!onmt-build-vocab --sentencepiece model_type=bpe --size 32000 --save_vocab sentencepiece_models/pvt_tgt pvt_tgt_data/pvt_tgt_train.txt

!onmt-build-vocab --sentencepiece model_type=bpe --size 32000 --save_vocab sentencepiece_models/src_tgt src_tgt_data/src_tgt_train.txt
!onmt-build-vocab --sentencepiece model_type=bpe --size 32000 --save_vocab sentencepiece_models/tgt_src src_tgt_data/tgt_src_train.txt

2021-12-11 13:12:12.918785: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-11 13:12:12.918837: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
sentencepiece_trainer.cc(177) LOG(INFO) Running command:  --model_type=bpe --vocab_size=32000 --input=/tmp/tmpd1w03nxn --model_prefix=sentencepiece_models/tgt
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /tmp/tmpd1w03nxn
  input_format: 
  model_prefix: sentencepiece_models/tgt
  model_type: BPE
  vocab_size: 32000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentence

trainer_interface.cc(466) LOG(INFO) all chars count=2020598378
trainer_interface.cc(477) LOG(INFO) Done: 99.9503% characters are covered.
trainer_interface.cc(487) LOG(INFO) Alphabet size=180
trainer_interface.cc(488) LOG(INFO) Final character coverage=0.999503
trainer_interface.cc(520) LOG(INFO) Done! preprocessed 22705199 sentences.
trainer_interface.cc(526) LOG(INFO) Tokenizing input sentences with whitespace: 22705199
trainer_interface.cc(537) LOG(INFO) Done! 3966034
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=38679200 min_freq=56055
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=11852128 size=20 all=9252 active=3301 piece=in
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=6201669 size=40 all=11927 active=5976 piece=is
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=4232549 size=60 all=14651 active=8700 piece=am
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=3295029 size=80 all=17302 active=11351 piece=il
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=2

bpe_model_trainer.cc(258) LOG(INFO) Added: freq=112984 size=1340 all=238947 active=16211 piece=demos
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=111488 size=1360 all=241367 active=18631 piece=▁esti
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=109719 size=1380 all=244053 active=21317 piece=▁diferentes
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=107665 size=1400 all=246765 active=24029 piece=echa
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=107648 min_freq=6151
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=106167 size=1420 all=248479 active=13923 piece=OM
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=103887 size=1440 all=251238 active=16681 piece=ulares
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=102183 size=1460 all=253416 active=18859 piece=▁Med
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=100239 size=1480 all=255666 active=21109 piece=▁ev
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=98384 size=1500 all=257919 active=23362 piece=▁Cons
b

bpe_model_trainer.cc(258) LOG(INFO) Added: freq=47747 size=2740 all=400050 active=24802 piece=int
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=47293 size=2760 all=402336 active=27088 piece=▁Pos
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=46893 size=2780 all=404989 active=29741 piece=portador
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=46335 size=2800 all=406887 active=31639 piece=ly
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=46327 min_freq=2687
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=45911 size=2820 all=410211 active=23076 piece=tario
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=45325 size=2840 all=412728 active=25593 piece=▁ru
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=44778 size=2860 all=414595 active=27460 piece=▁san
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=44472 size=2880 all=417158 active=30023 piece=▁limpieza
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=44099 size=2900 all=418770 active=31635 piece=álvula
bpe_model_t

bpe_model_trainer.cc(258) LOG(INFO) Added: freq=28491 size=4120 all=539722 active=29736 piece=ético
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=28277 size=4140 all=540969 active=30983 piece=▁algodón
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=28030 size=4160 all=543019 active=33033 piece=IB
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=27815 size=4180 all=544486 active=34500 piece=▁principalmente
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=27681 size=4200 all=547609 active=37623 piece=▁encuentran
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=27663 min_freq=1655
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=27457 size=4220 all=549894 active=29660 piece=▁Tab
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=27259 size=4240 all=551589 active=31355 piece=▁montaje
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=27155 size=4260 all=553227 active=32993 piece=▁aper
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=26984 size=4280 all=555117 active=34883 piec

...

bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=33 min_freq=17
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=33 size=26220 all=316806 active=15895 piece=▁spera
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=33 size=26240 all=316836 active=15925 piece=▁Coloro
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=33 size=26260 all=316847 active=15936 piece=▁disabi
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=33 size=26280 all=316871 active=15960 piece=▁vostra
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=33 size=26300 all=316869 active=15958 piece=▁Jenkins
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=33 min_freq=17
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=33 size=26320 all=316859 active=15834 piece=▁confuci
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=33 size=26340 all=316857 active=15832 piece=▁routine
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=33 size=26360 all=316840 active=15815 piece=▁adottati
bpe_model_trainer.cc(258) 

bpe_model_trainer.cc(258) LOG(INFO) Added: freq=31 size=27620 all=319649 active=15969 piece=▁Albuquerque
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=31 size=27640 all=319629 active=15949 piece=▁supervisore
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=31 size=27660 all=319611 active=15931 piece=▁raggiungibile
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=30 size=27680 all=319797 active=16117 piece=VII
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=30 size=27700 all=320041 active=16361 piece=▁FS
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=30 min_freq=16
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=30 size=27720 all=320185 active=16138 piece=onis
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=30 size=27740 all=320341 active=16294 piece=▁Das
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=30 size=27760 all=320467 active=16420 piece=Earth
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=30 size=27780 all=320576 active=16529 piece=▁1501
bpe_model_traine

bpe_model_trainer.cc(258) LOG(INFO) Added: freq=28 size=29040 all=323653 active=16420 piece=umati
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=28 size=29060 all=323662 active=16429 piece=▁1667
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=28 size=29080 all=323730 active=16497 piece=▁Itur
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=28 size=29100 all=323824 active=16591 piece=▁dira
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=28 min_freq=15
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=28 size=29120 all=323926 active=16285 piece=distan
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=28 size=29140 all=324012 active=16371 piece=▁Carte
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=28 size=29160 all=324063 active=16422 piece=▁Sando
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=28 size=29180 all=324081 active=16440 piece=▁climi
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=28 size=29200 all=324092 active=16451 piece=▁ribos
bpe_model_trainer.cc(167) LO

bpe_model_trainer.cc(258) LOG(INFO) Added: freq=26 size=30460 all=326950 active=16572 piece=▁June
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=26 size=30480 all=327014 active=16636 piece=▁Uffi
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=26 size=30500 all=327079 active=16701 piece=attiva
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=26 min_freq=14
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=26 size=30520 all=327165 active=16436 piece=vibile
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=26 size=30540 all=327208 active=16479 piece=▁Maure
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=26 size=30560 all=327254 active=16525 piece=▁bifor
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=26 size=30580 all=327303 active=16574 piece=▁subli
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=26 size=30600 all=327353 active=16624 piece=tivismo
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=26 min_freq=14
bpe_model_trainer.cc(258) LOG(INFO) Add

trainer_interface.cc(626) LOG(INFO) Saving vocabs: sentencepiece_models/tgt_src.vocab
INFO:tensorflow:Converting SentencePiece vocabulary to OpenNMT-tf format...


In [26]:
tokenize("src_tgt_test.txt","src_tgt_test")
tokenize("src_tgt_train.txt","src_tgt_train")
tokenize("src_tgt_val.txt","src_tgt_val")
tokenize("tgt_src_test.txt","tgt_src_test")
tokenize("tgt_src_train.txt","tgt_src_train")
tokenize("tgt_src_val.txt","tgt_src_val")

tokenize("src_test.txt","src_test")
tokenize("src_train.txt","src_train")
tokenize("src_val.txt","src_val")
tokenize("pvt_src_test.txt","pvt_src_test")
tokenize("pvt_src_train.txt","pvt_src_train")
tokenize("pvt_src_val.txt","pvt_src_val")

tokenize("pvt_tgt_test.txt","pvt_tgt_test")
tokenize("pvt_tgt_train.txt","pvt_tgt_train")
tokenize("pvt_tgt_val.txt","pvt_tgt_val")
tokenize("tgt_test.txt","tgt_test")
tokenize("tgt_train.txt","tgt_train")
tokenize("tgt_val.txt","tgt_val")

'tgt.txt'

In [8]:
config_src_pvt = {
    "model_dir": "src_pvt_model/",
    "data": {
        "train_features_file": "src_pvt_data/src_train.tok/",
        "train_labels_file": "src_pvt_data/pvt_src_train.tok/",
        "eval_features_file": "src_pvt_data/src_val.tok/",
        "eval_labels_file": "src_pvt_data/pvt_src_val.tok/",
        "source_vocabulary": "sentencepiece_models/src.vocab/",
        "target_vocabulary": "sentencepiece_models/pvt_src.vocab/",
    },
    "train": {
        "max_step": 25000,
        "save_checkpoints_steps": 500,
        "keep_checkpoint_max": 2,
    },
    "eval": {
        "save_eval_predictions": True,
        "steps": 50000,
        "max_exports_to_keep": 2,
        "early_stopping": {
            "metric": "loss",
            "min_improvement": 0.1,
            "steps": 100,
        },
    }
}

config_pvt_tgt = {
    "model_dir": "/content/pvt_tgt_model/",
    "data": {
        "train_features_file": "/content/pvt_tgt_data/pvt_tgt_train.tok/",
        "train_labels_file": "/content/pvt_tgt_data/tgt_train.tok/",
        "eval_features_file": "/content/pvt_tgt_data/pvt_tgt_val.tok/",
        "eval_labels_file": "/content/pvt_tgt_data/tgt_val.tok/",
        "source_vocabulary": "/content/sentencepiece_models/pvt_tgt.vocab/",
        "target_vocabulary": "/content/sentencepiece_models/tgt.vocab/",
    },
    "train": {
        "max_step": 25000,
        "save_checkpoints_steps": 500,
        "keep_checkpoint_max": 2,
    },
    "eval": {
        "save_eval_predictions": True,
        "steps": 50000,
        "max_exports_to_keep": 2,
        "early_stopping": {
            "metric": "loss",
            "min_improvement": 0.1,
            "steps": 100,
        },
    }
}

config_src_tgt = {
    "model_dir": "/content/src_tgt_model/",
    "data": {
        "train_features_file": "/content/src_tgt_data/src_tgt_train.tok",
        "train_labels_file": "/content/src_tgt_data/tgt_src_train.tok",
        "eval_features_file": "/content/src_tgt_data/src_tgt_val.tok",
        "eval_labels_file": "/content/src_tgt_data/tgt_src_val.tok",
        "source_vocabulary": "/content/sentencepiece_models/src_tgt.vocab",
        "target_vocabulary": "/content/sentencepiece_models/tgt_src.vocab",
    },
    "train": {
        "max_step": 25000,
        "save_checkpoints_steps": 500,
        "keep_checkpoint_max": 2,
    },
    "eval": {
        "save_eval_predictions": True,
        "steps": 50000,
        "max_exports_to_keep": 2,
        "early_stopping": {
            "metric": "loss",
            "min_improvement": 0.1,
            "steps": 100,
        },
    }
}

config_baseline = {
    "model_dir": "/content/baseline_model/",
    "data": {
        "train_features_file": "/content/src_tgt_data/src_tgt_train.tok/",
        "train_labels_file": "/content/src_tgt_data/tgt_src_train.tok/",
        "eval_features_file": "/content/src_tgt_data/src_tgt_val.tok/",
        "eval_labels_file": "/content/src_tgt_data/tgt_src_val.tok/",
        "source_vocabulary": "/content/sentencepiece_models/src_tgt.vocab/",
        "target_vocabulary": "/content/sentencepiece_models/tgt_src.vocab/",
    },
    "train": {
        "max_step": 25000,
        "save_checkpoints_steps": 500,
        "keep_checkpoint_max": 2,
    },
    "eval": {
        "save_eval_predictions": True,
        "steps": 50000,
        "max_exports_to_keep": 2,
        "early_stopping": {
            "metric": "loss",
            "min_improvement": 0.1,
            "steps": 100,
        },
    }
}

In [9]:
learning_rate = opennmt.schedules.NoamDecay(scale=2.0, model_dim=512, warmup_steps=8000)
optimizer = tfa.optimizers.LazyAdam(learning_rate)

2021-12-11 13:26:20.188249: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-11 13:26:20.188701: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-11 13:26:20.188745: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-3JSBOFT): /proc/driver/nvidia/version does not exist
2021-12-11 13:26:20.191701: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
# Training source-pivot model
src_pvt_model = opennmt.models.TransformerBase()
src_pvt_runner = opennmt.Runner(src_pvt_model, config_src_pvt, auto_config=True)
sp_config = src_pvt_runner._finalize_config(training=True)

src_pvt_runner.train(num_devices=1, with_eval=True)

INFO:tensorflow:Using OpenNMT-tf version 2.23.0
INFO:tensorflow:Using model:
(model): TransformerBase(
  (examples_inputter): SequenceToSequenceInputter(
    (features_inputter): WordEmbedder()
    (labels_inputter): WordEmbedder()
    (inputters): ListWrapper(
      (0): WordEmbedder()
      (1): WordEmbedder()
    )
  )
  (encoder): SelfAttentionEncoder(
    (position_encoder): SinusoidalPositionEncoder(
      (reducer): SumReducer()
    )
    (layer_norm): LayerNorm()
    (layers): ListWrapper(
      (0): SelfAttentionEncoderLayer(
        (self_attention): TransformerLayerWrapper(
          (layer): MultiHeadAttention(
            (linear_queries): Dense(512)
            (linear_keys): Dense(512)
            (linear_values): Dense(512)
            (linear_output): Dense(512)
          )
          (input_layer_norm): LayerNorm()
        )
        (ffn): TransformerLayerWrapper(
          (layer): FeedForwardNetwork(
            (inner): Dense(2048)
            (outer): Dense(512)
  

INFO:tensorflow:Using parameters:
data:
  eval_features_file: src_pvt_data/src_val.txt/
  eval_labels_file: src_pvt_data/pvt_src_val.txt/
  source_vocabulary: sentencepiece_models/src.vocab/
  target_vocabulary: sentencepiece_models/pvt_src.vocab/
  train_features_file: src_pvt_data/src_train.txt/
  train_labels_file: src_pvt_data/pvt_src_train.txt/
eval:
  batch_size: 32
  batch_type: examples
  early_stopping:
    metric: loss
    min_improvement: 0.1
    steps: 100
  length_bucket_width: 5
  max_exports_to_keep: 2
  save_eval_predictions: true
  steps: 50000
infer:
  batch_size: 32
  batch_type: examples
  length_bucket_width: 5
model_dir: src_pvt_model/
params:
  average_loss_in_time: true
  beam_width: 4
  decay_params:
    model_dim: 512
    warmup_steps: 8000
  decay_type: NoamDecay
  label_smoothing: 0.1
  learning_rate: 2.0
  num_hypotheses: 1
  optimizer: LazyAdam
  optimizer_params:
    beta_1: 0.9
    beta_2: 0.998
score:
  batch_size: 64
  batch_type: examples
  length_buc

2021-12-11 13:29:12.141066: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 2527594 of 31750988
2021-12-11 13:29:22.141071: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 5057119 of 31750988
2021-12-11 13:29:32.141073: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 7580650 of 31750988
2021-12-11 13:29:42.141072: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 10105716 of 31750988
2021-12-11 13:29:52.141068: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 12633059 of 31750988
2021-12-11 13:30:02.141069: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 15151844 of 31750988
2021-12-11 13:30:12.141075: I tensorflow/core/kernels/d

INFO:tensorflow:Number of model parameters: 93326081
INFO:tensorflow:Number of model weights: 260 (trainable = 260, non trainable = 0)
INFO:tensorflow:Saved checkpoint src_pvt_model/ckpt-1


KeyboardInterrupt: 

In [ ]:
# Training pivot-target model
pvt_tgt_model = opennmt.models.TransformerBase()
pvt_tgt_runner = opennmt.Runner(pvt_tgt_model, config_pvt_tgt, auto_config=True)
pt_config = pvt_tgt_runner._finalize_config(training=True)

pvt_tgt_runner.train(num_devices=1, with_eval=True)

In [ ]:
# Restore both models weights
src_pvt_model.initialize(data_config=sp_config['data'], params=sp_config['params'])
src_pvt_model.create_variables(optimizer=optimizer)

pvt_tgt_model.initialize(data_config=pt_config['data'], params=pt_config['params'])
pvt_tgt_model.create_variables(optimizer=optimizer)

checkpoint_path = sp_config['model_dir']
checkpoint = checkpoint_util.Checkpoint.from_config(sp_config, src_pvt_model, optimizer=optimizer)
checkpoint.restore(checkpoint_path=checkpoint_path, weights_only=True)

checkpoint_path = pt_config['model_dir']
checkpoint = checkpoint_util.Checkpoint.from_config(pt_config, pvt_tgt_model, optimizer=optimizer)
checkpoint.restore(checkpoint_path=checkpoint_path, weights_only=True)

count_weights(src_pvt_model)
count_weights(pvt_tgt_model)

In [ ]:
# Transfer weights to src_tgt_model
src_tgt_model = opennmt.models.TransformerBase()
src_tgt_runner = opennmt.Runner(src_tgt_model, config_src_tgt, auto_config=True)
st_config = src_tgt_runner._finalize_config(training=True)

src_tgt_model.initialize(data_config=st_config['data'], params=st_config['params'])
src_tgt_model.create_variables(optimizer=optimizer)

src_tgt_model.encoder = src_pvt_model.encoder
src_tgt_model.decoder = pvt_tgt_model.decoder

new_checkpoint = checkpoint_util.Checkpoint.from_config(st_config, src_tgt_model, optimizer=optimizer)
new_checkpoint.save()

In [7]:
# Training source-target model (using pretrained models)
src_tgt_model = opennmt.models.TransformerBase()
src_tgt_runner = opennmt.Runner(src_tgt_model, config_src_tgt, auto_config=True)
st_config = src_tgt_runner._finalize_config(training=True)
src_tgt_runner.train(num_devices=1, with_eval=True)

INFO:tensorflow:Using OpenNMT-tf version 2.19.0
INFO:tensorflow:Using model:
(model): TransformerBase(
  (examples_inputter): SequenceToSequenceInputter(
    (features_inputter): WordEmbedder()
    (labels_inputter): WordEmbedder()
    (inputters): ListWrapper(
      (0): WordEmbedder()
      (1): WordEmbedder()
    )
  )
  (encoder): SelfAttentionEncoder(
    (position_encoder): SinusoidalPositionEncoder(
      (reducer): SumReducer()
    )
    (layer_norm): LayerNorm()
    (layers): ListWrapper(
      (0): SelfAttentionEncoderLayer(
        (self_attention): TransformerLayerWrapper(
          (layer): MultiHeadAttention(
            (linear_queries): Dense(512)
            (linear_keys): Dense(512)
            (linear_values): Dense(512)
            (linear_output): Dense(512)
          )
          (input_layer_norm): LayerNorm()
        )
        (ffn): TransformerLayerWrapper(
          (layer): FeedForwardNetwork(
            (inner): Dense(2048)
            (outer): Dense(512)
  

In [ ]:
# Training source-target model (using no models)
baseline_model = opennmt.models.TransformerBase()
baseline_runner = opennmt.Runner(baseline_model, config_baseline, auto_config=True)

baseline_runner.train(num_devices=1, with_eval=True)

In [ ]:
# Compute scores
baseline_scores = compute_scores(
    runner=baseline_runner,
    features_filename="/content/src_tgt_data/src_tgt_test.txt",
    labels_filename="/content/src_tgt_data/tgt_src_test.txt",
    pred_filename="/content/baseline_pred.txt")

pivot_based_tl_scores = compute_scores(
    runner=src_tgt_runner,
    features_filename="/content/src_tgt_data/src_tgt_test.txt",
    labels_filename="/content/src_tgt_data/tgt_src_test.txt",
    pred_filename="/content/src_to_tgt_pred.txt")

print(f"============ Baseline Source-Target NMT Evaluation ============\n {baseline_scores}")
print(f"============ Pretrain Source-Target NMT Evaluation ============\n {pivot_based_tl_scores}")